## **INSTALL LIBRARY**

In [ ]:
pip install plotly
pip install pyodbc
pip install dash

## **IMPORT LIBRARY**

In [2]:
from dash.dependencies import Input, Output
from dash import Dash, html, dcc
import pandas as pd
import plotly.express as px
import pyodbc

## **PROCESSING DASHBOARD**

### **Fetch Data**

In [3]:
def fetch_data(query, connection):
    """
    Truy vấn dữ liệu từ cơ sở dữ liệu.
    """
    try:
        df = pd.read_sql(query, connection)
        print(f"Truy vấn dữ liệu thành công: {query[:30]}...")
        return df
    except Exception as e:
        print("Lỗi truy vấn dữ liệu:", e)
        return None

### **Stacked Bar Chart**

In [4]:
def create_stacked_bar_chart(df):
    """
    Tạo biểu đồ cột chồng từ dữ liệu.
    """
    try:
        fig = px.bar(
            df,
            x='GroupMajor',
            y='Count_GroupMajor',
            color='CAP_BAC',
            title='Biểu đồ cột chồng: Count của GroupMajor theo CAP_BAC',
            labels={'GroupMajor': 'Nhóm ngành', 'Count_GroupMajor': 'Số lượng', 'CAP_BAC': 'Cấp bậc'}
        )
        return fig
    except Exception as e:
        print("Lỗi tạo biểu đồ cột chồng:", e)
        return None

### **Donut Chart**

In [5]:
def create_donut_chart(df):
    """
    Tạo biểu đồ donut từ dữ liệu.
    """
    try:
        fig = px.pie(
            df,
            names='Location',
            values='Count_GroupMajor',
            title='Biểu đồ Donut: Số lượng GroupMajor theo Location',
            labels={'Location': 'Vị trí', 'Count_GroupMajor': 'Số lượng'}
        )
        fig.update_traces(hole=0.4)
        return fig
    except Exception as e:
        print("Lỗi tạo biểu đồ donut:", e)
        return None


### **Treemap Chart**

In [6]:
def create_treemap_chart(df):
    """
    Tạo biểu đồ treemap từ dữ liệu.
    """
    try:
        fig = px.treemap(
            df,
            path=['GroupMajor'],
            values='Count_GroupMajor',
            title='Biểu đồ Treemap: Số lượng GroupMajor',
            labels={'GroupMajor': 'Nhóm ngành', 'Count_GroupMajor': 'Số lượng'}
        )
        fig.update_traces(textinfo='label+value')
        return fig
    except Exception as e:
        print("Lỗi tạo biểu đồ treemap:", e)
        return None


### **Line Chart**

In [7]:
def create_line_chart(df):
    """
    Tạo biểu đồ đường từ dữ liệu với hai trục y.
    """
    try:
        fig = px.line(
            df,
            x='GroupMajor',
            y=['AVR_MinSalary', 'AVR_MaxSalary'],
            title='Biểu đồ đường: Mức lương tối thiểu và tối đa theo GroupMajor',
            labels={'GroupMajor': 'Nhóm ngành', 'value': 'Mức lương', 'variable': 'Loại lương'}
        )
        return fig
    except Exception as e:
        print("Lỗi tạo biểu đồ đường:", e)
        return None


### **Slicer Chart and Layout**

In [8]:
def create_dash_app_with_slicer(fig_bar, fig_donut, fig_treemap, fig_line, df_bar, df_donut, df_line):
    """
    Tạo ứng dụng Dash với chức năng slicer dựa trên CAP_BAC và Location.
    """
    cap_bac_options = [{'label': cap_bac, 'value': cap_bac} for cap_bac in df_bar['CAP_BAC'].unique()]
    location_options = [{'label': location, 'value': location} for location in df_bar['Location'].unique()]

    app = Dash(__name__, title='Phân tích tuyển dụng tại 3 tỉnh thành lớn ở Việt Nam')
    app.layout = html.Div(
        children=[
            html.H1(
                "Tuyển dụng tại 3 tỉnh thành lớn",
                style={
                    'textAlign': 'center',  # Căn giữa
                    'color': '#007bff',  # Màu chữ
                    'fontSize': '36px',  # Kích thước chữ
                    'marginBottom': '20px',  # Khoảng cách bên dưới
                    'border': '2px solid #007bff',  # Viền khung
                    'padding': '10px',  # Khoảng cách giữa chữ và viền
                    'borderRadius': '10px',  # Bo góc khung
                    'width': '50%',  # Độ rộng khung
                    'margin': '0 auto',  # Căn giữa khung
                    'backgroundColor': '#f8f9fa'  # Nền nhạt cho khung
                }
            ),
            html.Div(
                children=[
                    html.Label("Chọn cấp bậc:"),
                    dcc.Dropdown(
                        id='slicer-dropdown-capbac',
                        options=cap_bac_options,
                        value=None,
                        placeholder="Chọn cấp bậc để lọc...",
                        multi=False
                    ),
                    html.Label("Chọn vị trí:"),
                    dcc.Dropdown(
                        id='slicer-dropdown-location',
                        options=location_options,
                        value=None,
                        placeholder="Chọn vị trí để lọc...",
                        multi=False
                    )
                ],
                style={'width': '30%', 'margin': '10px auto'}
            ),
            html.Div(
                children=[
                    html.Div(
                        dcc.Graph(id='stacked-bar-chart', figure=fig_bar),
                        style={'width': '48%', 'display': 'inline-block'}
                    ),
                    html.Div(
                        dcc.Graph(id='donut-chart', figure=fig_donut),
                        style={'width': '48%', 'display': 'inline-block'}
                    )
                ],
                style={'display': 'flex', 'justify-content': 'space-between'}
            ),
            html.Div(
                children=[
                    html.Div(
                        dcc.Graph(id='treemap-chart', figure=fig_treemap),
                        style={'width': '48%', 'display': 'inline-block'}
                    ),
                    html.Div(
                        dcc.Graph(id='line-chart', figure=fig_line),
                        style={'width': '48%', 'display': 'inline-block'}
                    )
                ],
                style={'display': 'flex', 'justify-content': 'space-between', 'margin-top': '20px'}
            )
        ]
    )

    @app.callback(
        [
            Output('stacked-bar-chart', 'figure'),
            Output('donut-chart', 'figure'),
            Output('treemap-chart', 'figure'),
            Output('line-chart', 'figure')
        ],
        [
            Input('slicer-dropdown-capbac', 'value'),
            Input('slicer-dropdown-location', 'value')
        ]
    )
    def update_charts(selected_cap_bac, selected_location):
        filtered_df_bar = df_bar.copy()
        filtered_df_donut = df_donut.copy()
        filtered_df_line = df_line.copy()

        if selected_cap_bac:
            filtered_df_bar = filtered_df_bar[filtered_df_bar['CAP_BAC'] == selected_cap_bac]
            filtered_df_donut = filtered_df_donut[filtered_df_donut['CAP_BAC'] == selected_cap_bac]

        if selected_location:
            filtered_df_bar = filtered_df_bar[filtered_df_bar['Location'] == selected_location]
            filtered_df_donut = filtered_df_donut[filtered_df_donut['Location'] == selected_location]
            filtered_df_line = filtered_df_line[filtered_df_line['GroupMajor'].isin(filtered_df_bar['GroupMajor'])]

        bar_chart = create_stacked_bar_chart(filtered_df_bar)
        donut_chart = create_donut_chart(filtered_df_donut)
        treemap_chart = create_treemap_chart(filtered_df_bar)
        line_chart = create_line_chart(filtered_df_line)

        return bar_chart, donut_chart, treemap_chart, line_chart

    return app


### **Call Function, Connect and Querry**

In [9]:
def main():
    """
    Hàm chính để chạy ứng dụng.
    """
    server = 'LAPTOP-KS80S7GT\\DATANQTHY'
    database = 'DatabaseofGroup5'
    driver = '{ODBC Driver 17 for SQL Server}'

    connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
    conn = pyodbc.connect(connection_string)

    stacked_bar_query = """
    SELECT 
        LUONG.GroupMajor, 
        LUONG.Location,
        NHANVIEN.CAP_BAC, 
        COUNT(LUONG.GroupMajor) AS Count_GroupMajor
    FROM LUONG
    JOIN NHANVIEN ON LUONG.Location = NHANVIEN.Location
    GROUP BY LUONG.GroupMajor, LUONG.Location, NHANVIEN.CAP_BAC
    """

    donut_query = """
    SELECT 
        LUONG.Location, 
        NHANVIEN.CAP_BAC, 
        COUNT(LUONG.GroupMajor) AS Count_GroupMajor
    FROM LUONG
    JOIN NHANVIEN ON LUONG.Location = NHANVIEN.Location
    GROUP BY LUONG.Location, NHANVIEN.CAP_BAC
    """

    line_chart_query = """
    SELECT 
        LUONG.GroupMajor, 
        AVG(LUONG.AVR_MinSalary) AS AVR_MinSalary,
        AVG(LUONG.AVR_MaxSalary) AS AVR_MaxSalary
    FROM LUONG
    GROUP BY LUONG.GroupMajor
    """

    df_bar = fetch_data(stacked_bar_query, conn)
    df_donut = fetch_data(donut_query, conn)
    df_line = fetch_data(line_chart_query, conn)

    conn.close()

    if df_bar is not None and df_donut is not None and df_line is not None:
        fig_bar = create_stacked_bar_chart(df_bar)
        fig_donut = create_donut_chart(df_donut)
        fig_treemap = create_treemap_chart(df_bar)
        fig_line = create_line_chart(df_line)

        app = create_dash_app_with_slicer(fig_bar, fig_donut, fig_treemap, fig_line, df_bar, df_donut, df_line)
        app.run_server(debug=True)

if __name__ == "__main__":
    main()


C:\Users\HP\AppData\Local\Temp\ipykernel_20440\109793085.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Truy vấn dữ liệu thành công: 
    SELECT 
        LUONG.Gro...
Truy vấn dữ liệu thành công: 
    SELECT 
        LUONG.Loc...
Truy vấn dữ liệu thành công: 
    SELECT 
        LUONG.Gro...
